<a href="https://colab.research.google.com/github/kengustafson/DeepDive-DataScience/blob/main/Solar_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import pickle
import plotly.express as px
import plotly.graph_objects as go
import requests

In [29]:
def pred_solar_power(solar_df, station_rated_output):
    """Accepts dataframe with Global Horizontal Irradiance (floats) at a certain location in W/m^2,
    Scaler indicating output of plant in MW.
    ?Returns DataFrame of 'Month' (1-12), output power ('MW'), in whatever timeframe submitted
    """

    ghi = solar_df   # GHI in W/m^2

    pred_power = ghi.div(800).mul(station_rated_output).rename('MW')

    return pred_power.sum()

    #return pd.concat([solar_df, pred_power], axis=1)

In [13]:
psm_url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.json?api_key=0WIF3emAV5iUrIyfYwk0Fcm5wUZGfwLVOlLD7Nc1'
params_north   = '&wkt=POLYGON((-103.072391+36.98%2C-109.014299+36.98%2C-109.007507+35.7%2C-103.131484+35.7%2C-103.072391+36.98))'
params_upper   = '&wkt=POLYGON((-103.131484+35.7%2C-109.007507+35.7%2C-109.007507+34.45%2C-103.131484+34.45%2C-103.131484+35.7))'
params_lower   = '&wkt=POLYGON((-103.131484+34.45%2C-109.007507+34.45%2C-109.007507+33.20%2C-103.131484+33.20%2C-103.131484+34.45))'
params_south   = '&wkt=POLYGON((-103.131484+33.20%2C-109.007507+33.20%2C-109.011065+32.066178%2C-103.128003+32.061526%2C-103.131484+33.20))'
params_bootheel= '&wkt=POLYGON((-106.75+32.07%2C-109.01+32.07%2C-109.01+31.35%2C-108.22+31.35%2C-108.22+31.8%2C-106.6+31.8%2C-106.75+32.07))'

attributes = '&attributes=ghi&names=2019&utc=false&leap_day=false&interval=60&email=kengustafson@gmail.com'

psm_res = requests.get(psm_url + params_upper + attributes)

content = {}

if psm_res.status_code == 200:
    print(psm_res.headers['content-type'])

    content = psm_res.content

else:
    print('ERROR')

application/json; charset=utf-8


In [ ]:
psm_res.json()

Done - North -  'outputs': {'downloadUrl': 'https://mapfiles.nrel.gov/data/solar/fd265c4265cf7f884c722586ff6456c9.zip',

Done - Lower -  'outputs': {'downloadUrl': 'https://mapfiles.nrel.gov/data/solar/0281b52ed34c94cdbb7f0db9a7beb0f3.zip',


Done - South -  'outputs': {'downloadUrl': 'https://mapfiles.nrel.gov/data/solar/b7aea2b63388f4cdce11f22b6dc279e6.zip',


Done - Bootheel -  'outputs': {'downloadUrl': 'https://mapfiles.nrel.gov/data/solar/ee0114c6233e04725e53588e2c15c6e1.zip',

Done - Upper -  'outputs': {'downloadUrl': 'https://mapfiles.nrel.gov/data/solar/ef11a546199f97973e31e321638a961c.zip',



In [4]:
file = '/content/drive/MyDrive/Data Science Student Material/Capstone Project/Data/all_solar.p'
dict_allData_read = pickle.load(open(file,'rb'))
solar_df = pd.DataFrame.from_dict(dict_allData_read)
del dict_allData_read

In [ ]:
solar_df

In [34]:
power_df = pd.DataFrame(columns=['lat', 'lon', 'power'])

In [35]:
for el in solar_df.columns[5:]:
    tmp_df = pd.DataFrame({'lat': el.split('_')[0], 'lon': el.split('_')[1], 'power': pred_solar_power(solar_df[el], 10)}, index=[0])
    power_df = pd.concat([power_df, tmp_df], axis=0, ignore_index=True)

In [42]:
"""
scl = [0,"rgb(150,0,90)"],[0.07,"rgb(0, 33, 200)"],[0.12,"rgb(0, 75, 255)"],\
[0.22,"rgb(0, 152, 255)"],[0.3,"rgb(44, 255, 150)"],[0.4,"rgb(151, 255, 0)"],\
[0.5,"rgb(255, 234, 0)"],[0.85,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]
"""
scl = [0,"rgb(0,75,140)"],[0.07,"rgb(0, 75, 140)"],[0.12,"rgb(0, 75, 140)"],\
[0.22,"rgb(0, 152, 140)"],[0.3,"rgb(44, 255, 80)"],[0.4,"rgb(151, 255, 0)"],\
[0.5,"rgb(255, 234, 0)"],[0.85,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

fig = go.Figure(data=go.Scattergeo(
    lat = power_df['lat'],
    lon = power_df['lon'],
    text = power_df['power'].round(0).astype(str) + ' MegaWatts',
    
    marker = dict(
        color = round(power_df['power'], 0),
        colorscale = scl,
        reversescale = False,
        opacity = 0.7,
        size = 3,
        colorbar = dict(
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            showticksuffix = "last",
            dtick = 500
        ),
        colorbar_title="Yearly Output(MW)/10MW Array"
    )
))

fig.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='New Mexico Potential Solar Production',
)
fig.show()


In [40]:
m1 = go.scattergeo.Marker(color='#000000', size=7, symbol='square', showscale=False)
wlats = [33.453056, 33.45, 32.57, 35.016111, 35.353889, 36.466667, 32.571944, 34.982778, 35.08, 35.634444, 31.8, 32.251667, 35.018333, 32.681944, 32.345556, 32.1275, 32.183611, 32.469722, 34.986111, 34.5825, 35.104722, 34.831111, 34.741111, 34.692222, 35.561389, 34.81, 34.746667, 32.976389, 35.522222, 32.626111, 35.0492, 36.716944, 35.875556]
wlons = [-104.461111, -104.456111, -107.48, -106.098056, -106.858056, -104.633333, -107.486389, -103.378333, -106.865, -105.178333, -106.666667, -106.913333, -106.855556, -103.303611, -104.234722, -103.137778, -103.183056, -103.323333, -106.736111, -106.683056, -106.4825, -106.761944, -106.655, -106.688056, -106.086111, -106.52, -106.654444, -105.971111, -108.778889, -107.258889, -106.53, -105.609444, -106.31]

In [41]:
fig.add_scattergeo(lat=wlats, lon=wlons, marker=m1, showlegend=False)